# Kapitel 13 - 03

> ### <span style="color:red">ACHTUNG:</span> Bitte zum Starten im Menü `Cell->Run All` ausführen.

## Deaktivieren der Warnungen

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Erträumte Bilder

#### Listing 13.28

In [ ]:
import numpy as np

# IPython.display brauchen wir um die Ausgabe einer 
# Zelle zu löschen – sehr praktisch
from IPython.display import clear_output

import matplotlib as mpl
from matplotlib import pyplot as plt

from tensorflow.keras.preprocessing import image
import tensorflow as tf



#### Listing 13.29

In [ ]:
# Normalisiere das Bild
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

# Bildanzeige
def show(img):
  plt.figure(figsize=(12,12))
  plt.grid(False)
  plt.axis('off')
  plt.imshow(img)
  plt.show()

#### Listing 13.30

In [ ]:
import os

basedir = 'deepdream'
original_img_file = os.path.join(basedir,'mallorca.jpg')

# wir laden das Bild und verkleinern es etwas zur schnelleren Berechnung
original_img = tf.keras.preprocessing.image.load_img(original_img_file, target_size=[375,275])

original_img = np.array(original_img)

show(original_img)


#### Listing 13.31

In [ ]:
base_model = InceptionV3(include_top=False, weights='imagenet')
base_model.summary()

#### Listing 13.32

In [ ]:
# Ziel ist die Maximierung der Aktivierung in diesen Layern
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

# Unser Traummodell basiert auf Inception_v3
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)


#### Listing 13.33

In [ ]:
def calculate_loss(img, model):
    # Vorwärts Pass des Bilders durch das Modell um die Aktivierung zu erhalten
    # Konvertierung des Bildes zu einer Batchgrösse 1
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    
    losses = []
    for act in layer_activations:
        loss = tf.math.reduce_mean(act)
        losses.append(loss)
        
    return  tf.reduce_sum(losses)

#### Listing 13:34

In [ ]:
@tf.function
def deepdream(model, img, step_size):
    with tf.GradientTape() as tape:
      # Gradientenberechnung in Bezug auf das Bild `img`
      tape.watch(img)
      loss = calculate_loss(img, model)

    # Berechne den Gradienten der Loss in Bezug auf die Pixel des Eingabebildes.
    gradients = tape.gradient(loss, img)

    # Normalisierung des Gradienten.
    gradients /= tf.math.reduce_std(gradients) + 1e-8 
    
    # Im Gradientenanstieg wird der "Loss" maximiert, damit das Eingabild die Layers zu Traummustern "anregt"
    # Das Bild wird direkt upgedatet indem de Gradient dazuaddiert wird (haben die gleiche Dimension)
    img = img + gradients*step_size
    img = tf.clip_by_value(img, -1, 1)
    return loss, img

#### Listing 13.35
Decorator-Funktion

In [ ]:
import time

# das ist unsere Decorator-Funktion
def timeit(original_fn):
    def decorator_fn(*args, **kwargs):
        start = time.time()
        res = original_fn(*args, **kwargs)
        end = time.time()
        print('func:%r args:[%r, %r] Ergebnis: %r Zeit: %2.6f sec' % (original_fn.__name__,
                                                         args, kwargs, res, end - start))
 
    return decorator_fn
 
@timeit
def add(x, y):
    return x + y

add(3,4)

#### Listing 13.36

In [ ]:
def run_deepdream_feedback(model, img, steps=100, step_size=0.01):
  # Konvertierung von uint8 in den Bereich das von Inception_v3 erwartet wird.
  img = tf.keras.applications.inception_v3.preprocess_input(img)

  for step in range(steps):
    loss, img = deepdream(model, img, step_size)
    
    if step % 100 == 0:
      clear_output(wait=True)
      show(deprocess(img))
      print ("Step {}, loss {}".format(step, loss))


  result = deprocess(img)
  clear_output(wait=True)
  show(result)

  return result



#### Listing 13.37

In [ ]:
OCTAVE_SCALE = 1.3

img = tf.constant(np.array(original_img))

base_shape = tf.cast(tf.shape(img)[:-1], tf.float32)

for n in range(3):
  new_shape = tf.cast(base_shape*(OCTAVE_SCALE**n), tf.int32)

  img = tf.image.resize(img, new_shape).numpy()

  img = run_deepdream_feedback(model=dream_model, img=img, steps=500, step_size=0.01)